In [ ]:
import os

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.auto import tqdm

from datasets.utils import PreprocessingDataset
from models.utils import get_model_by_name
from utils.environment import modified_environ


# Parameters
BATCH_SIZE, NUM_WORKERS = 8, 4
# IMAGES_DIR = os.path.join("/", "mnt", "workspace", "Ugallery", "images")
IMAGES_DIR = os.path.join("/", "mnt", "data2", "pinterest_iccv", "images")
IMAGES_EXT = ["*.gif", "*.jpg", "*.png", "*.webp"]
# OUTPUT_EMBEDDING_PATH = os.path.join("data", "UGallery", "ugallery_embedding.npy")
OUTPUT_EMBEDDING_PATH = os.path.join("data", "Pinterest", "pinterest_embedding.npy")
MODELS = [
    "resnet50",
    "resnext101_32x8d",
]
USE_GPU = True


How to concatenate embeddings parts:

```python
# Load parts
part1 = np.load(PART_1_PATH, allow_pickle=True)
part2 = np.load(PART_2_PATH, allow_pickle=True)
# Concatenate parts into a single embedding
embedding = np.concatenate((part1, part2), axis=0)
# Save embedding to file
np.save(OUTPUT_PATH, embedding, allow_pickle=True)
```

In [ ]:
import PIL


# Needed for some images in Pinterest dataset
PIL.Image.MAX_IMAGE_PIXELS = 1_000_000_000

In [ ]:
%%time
# Setting up torch device (useful if GPU available)
print("\nCreating device...")
device = torch.device("cuda:0" if torch.cuda.is_available() and USE_GPU else "cpu")
if torch.cuda.is_available() != USE_GPU:
    print((f"\nNotice: Not using GPU - "
           f"Cuda available ({torch.cuda.is_available()}) "
           f"does not match USE_GPU ({USE_GPU})"
    ))

# Downloading models for feature extraction
print("\nDownloading models...")
with modified_environ(TORCH_HOME="."):
    PRETRAINED_MODELS = dict()
    for model_name in MODELS:
        print(f"Model: {model_name}")
        model = get_model_by_name(model_name).eval().to(device)
        PRETRAINED_MODELS[model_name] = model

# Setting up transforms and dataset
print("\nSetting up transforms and dataset...")
images_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image_dataset = PreprocessingDataset(
    IMAGES_DIR,
    extensions=IMAGES_EXT,
    transform=images_transforms,
)
image_dataloader = DataLoader(image_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
print(f">> Images dataset: {len(image_dataset)}")

# Calculate embedding dimension size
emb_dim = 0
for model in PRETRAINED_MODELS.values():
    dummy_input = torch.ones(1, *image_dataset[0]["image"].size()).to(device)
    dummy_output = model(dummy_input)
    emb_dim += dummy_output.size(1)
print(f">> Embedding dimension size: {emb_dim}")

# Feature extraction phase
print(f"\nFeature extraction...")
output_ids = np.empty(len(image_dataset), dtype=object)
output_embedding = torch.zeros((len(image_dataset), emb_dim), dtype=torch.float32, device=device)
with torch.no_grad():
    batch_features = torch.tensor((BATCH_SIZE, emb_dim), dtype=torch.float32, device=device)
    for batch_i, sample in enumerate(tqdm(image_dataloader, desc="Feature extraction")):
        item_image = sample["image"].to(device)
        item_idx = sample["idx"]
        output_ids[[*item_idx]] = sample["id"]
        features = list()
        for model in PRETRAINED_MODELS.values():
            output = model(item_image).squeeze(-1).squeeze(-1)
            features.append(output)
        torch.cat(features, dim=1, out=batch_features)
        output_embedding[item_idx] = batch_features
output_embedding = output_embedding.cpu().numpy()

# Fill output embedding
embedding = np.ndarray(
    shape=(len(image_dataset), 2),
    dtype=object,
)
for i in range(len(image_dataset)):
    embedding[i] = np.asarray([output_ids[i], output_embedding[i]])
print(f">> Embedding shape: {embedding.shape}")

# Save embedding to file
print(f"\nSaving embedding to file... ({OUTPUT_EMBEDDING_PATH})")
np.save(OUTPUT_EMBEDDING_PATH, embedding, allow_pickle=True)

# Free some memory
if USE_GPU:
    print(f"\nCleaning GPU cache...")
    for name, model in PRETRAINED_MODELS.items():
        PRETRAINED_MODELS[name] = model.to(torch.device("cpu"))
    torch.cuda.empty_cache()

# Finished
print("\nDone")
